In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, Binarizer,KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

import pandas as pd

In [2]:
breast = fetch_ucirepo(id=15)

In [3]:
X = breast.data.features 
y = breast.data.targets
df = X
df["malign"] = np.array(y==4, dtype=int)

display(df.shape)
df = df.dropna()
display(df.shape)

print(cross_val_score(LogisticRegression(),df.loc[:, df.columns != 'malign'],df.malign).mean())

df.columns

/tmp/ipykernel_93947/3258228037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["malign"] = np.array(y==4, dtype=int)


(699, 10)

(683, 10)

0.9663696865607557


Index(['Clump_thickness', 'Uniformity_of_cell_size',
       'Uniformity_of_cell_shape', 'Marginal_adhesion',
       'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin',
       'Normal_nucleoli', 'Mitoses', 'malign'],
      dtype='object')

In [4]:
df

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses,malign
0,5,1,1,1,2,1.0,3,1,1,0
1,5,4,4,5,7,10.0,3,2,1,0
2,3,1,1,1,2,2.0,3,1,1,0
3,6,8,8,1,3,4.0,3,7,1,0
4,4,1,1,3,2,1.0,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2.0,1,1,1,0
695,2,1,1,1,2,1.0,1,1,1,0
696,5,10,10,3,7,3.0,8,10,2,1
697,4,8,6,4,3,4.0,10,6,1,1


In [5]:
column_trans = ColumnTransformer([
    ('label', "passthrough", ["malign"]),
    ('bin', KBinsDiscretizer(n_bins=2, encode="ordinal", strategy="kmeans") , ['Clump_thickness', 'Uniformity_of_cell_size',
       'Uniformity_of_cell_shape', 'Marginal_adhesion',
       'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin',
       'Normal_nucleoli', 'Mitoses']),
],
verbose_feature_names_out=True)

column_trans.fit(df)

cols = column_trans.get_feature_names_out()
df_out = pd.DataFrame(column_trans.transform(df), columns=cols, dtype=int)
len(cols), cols, df_out.shape

/home/sheid/.cache/pypoetry/virtualenvs/ijar-experiments-psl-vs-slim-s9rquOQ_-py3.11/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


(10,
 array(['label__malign', 'bin__Clump_thickness',
        'bin__Uniformity_of_cell_size', 'bin__Uniformity_of_cell_shape',
        'bin__Marginal_adhesion', 'bin__Single_epithelial_cell_size',
        'bin__Bare_nuclei', 'bin__Bland_chromatin', 'bin__Normal_nucleoli',
        'bin__Mitoses'], dtype=object),
 (683, 10))

In [6]:
df_out.to_csv("../breast.csv", index=False)
print(cross_val_score(LogisticRegression(),df_out.loc[:, df_out.columns != 'label__malign'],df_out.label__malign).mean())
df_out.describe()

0.9546586517818806


,label__malign,bin__Clump_thickness,bin__Uniformity_of_cell_size,bin__Uniformity_of_cell_shape,bin__Marginal_adhesion,bin__Single_epithelial_cell_size,bin__Bare_nuclei,bin__Bland_chromatin,bin__Normal_nucleoli,bin__Mitoses
count,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,0.349927,0.267936,0.212299,0.221083,0.172767,0.153734,0.254758,0.203514,0.199122,0.058565
std,0.477296,0.443208,0.409235,0.415281,0.378323,0.360957,0.436045,0.402907,0.399632,0.234981
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
